In [8]:
# For Raw Image Python3.8
import rawpy
#import rawpy.enhance
import glob
import os
import numpy as np
import tifffile as tif

# 進捗の確認に使用
import time
from tqdm import tqdm

In [9]:
# ============================================================================
# 対象観測シーンIDとデータディレクトリ
SceneID = "Kotogahama-20240327-1302-P1"
LensSID = "P1-20241224-lensShadingImages"

# 入出力ファイルへパスを通す
Win  = "/mnt/i/"
Mac  = "/Volumes/RM1/"
Lin  = "/media/kut/RM1/"
OSd  = Lin
Dir1 = OSd  + "003_RESEARCH-河川計測_4_BRF/" +SceneID+ "/"
Dir2 = Dir1 + "O_IntermediateData/"
Dir3 = Dir1 + "I_Raw/"
Dir4 = Dir1 + "O_Tif/"
Dir9 = OSd  + "003_RESEARCH-河川計測_4_BRF/" +LensSID+ "/"


# 入力ファイル
IFile1 = Dir1 +SceneID+ "_ListImages.csv"         # 入力ファイル１：原画像のリスト
IFile2 = Dir9 +"P1-FlatCofficient.npz"             # 入力ファイル２：Lens Shading Cofficient

# 原画像全部処理したいときはこれ
PhotoIDs = []
with open(IFile1, "r", encoding="utf-8") as f: # 画像データを読み込み
    Photolist = f.readlines()
for i in Photolist:
    PhotoIDs.append(i.rstrip())

# レンズシェーディング補正係数の読み込み
FlatCo_npz     = np.load(IFile2)
FlatCofficient = FlatCo_npz["arr_0"]

In [10]:
np.min(FlatCofficient)

0.3010820247676237

# Tiff 形式で保存する

In [11]:
# クラスメソッド：イメージ毎のカメラ情報など      
class ImageParameterEach: # 変数はPhotoIDに紐ずいたカメラデータから自動で読み取られる
    def __init__(self, photo):
        
        self.OFile1 = Dir4 + photo + ".tif"
        
        self.raw_original = rawpy.imread(Dir3+photo+".DNG")

        # ディベイヤ処理する
        self.RGR_original = self.raw_original.postprocess(
                        demosaic_algorithm = rawpy.DemosaicAlgorithm.LINEAR,  # ここで処理方法を指定
                        output_color       = rawpy.ColorSpace.raw,            # rawは無変換
                        no_auto_bright     = True,                            # 自動明るさ補正をキャンセル
                        use_camera_wb      = False,                           # カメラが記録したホワイトバランスを使用
                        user_wb            = [2.546875, 1.0, 1.83203125, 0.0],# Measurement using Light box
                        output_bps         = 16,                              # 16bitにする
                        no_auto_scale      = False,                           # 16bitに正規化する
                        gamma              = [1,0]
                        )
        self.CorrectedImage1 = np.round(self.RGR_original / FlatCofficient)
        self.CorrectedImage2 = np.where(self.CorrectedImage1 > 65535, 65535, self.CorrectedImage1).astype(np.uint16)

In [12]:
# RAWPYによるTIFF保存
for i in tqdm(PhotoIDs):
    photo = ImageParameterEach(i)
    tif.imwrite(photo.OFile1, photo.CorrectedImage2.astype('uint16'))

100%|██████████| 36/36 [02:25<00:00,  4.04s/it]


print(np.min(photo.RGR_original))
FlatCofficient[:,:,0]

# Line に通知
import Tool_LineNotify
Tool_LineNotify.LineNotify(SceneID + "\n"+ "Finished Raw2Tif")